In [2]:
# # installations to avoid import errors with fiftyone
!pip install -U Jinja2
!pip install opencv-python-headless==4.1.2.30

# fiftyone 
!pip install fiftyone
import fiftyone as fo 

# Load the drive to access the images and annotations
from google.colab import drive
drive.mount('/content/gdrive')

# reduce the path of the drive
def driveSymboLink():
  # this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
  !ln -s /content/gdrive/My\ Drive/ /mydrive
  # !ls /mydrive

driveSymboLink()

# pandas/numpy/matplotlib imports
import shutil
import pandas as pd
import json
import matplotlib.pyplot as plt
import os
import numpy as np
from PIL import Image, ImageOps


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 10.0 MB/s 
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 1.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 7.9 MB/s 
     |███████████

To run crop/resize, specify the confidence threshold in the cropped function, and the WDIR for cropped images in detect_train and detect_val.

In [13]:
def cropped(paths, weights):
    for path in paths:
      if 'train' in path:
        detect = '/content/gdrive/MyDrive/Animal_Identification/yolov5/detect_train.py'
      else:
        detect = '/content/gdrive/MyDrive/Animal_Identification/yolov5/detect_val.py'
      cropped = path + '/cropped_images/'
      imgs= path + '/images/'

      !mkdir $cropped
      !python $detect --source $imgs --weights $weights --conf 0.85 --cropped_folder_root $cropped_path

def resized(paths, head_size=(64,64), flank_size=(64,64), full_size=(96,128)):
  for path in paths:
    resized = path + '_resized_90/'
    !mkdir $resized
    cropped_path = path + 'cropped_images_90/'
    for img_dir in os.listdir(cropped_path):
      full_image_path = cropped_path + '/' + img_dir
      if 'jpg' not in full_image_path:
        new_img_dir = resized + '/' + img_dir
        !mkdir $new_img_dir
        for obj_dir in os.listdir(cropped_path + '/' + img_dir):
          for img_path in os.listdir(cropped_path + '/' + img_dir + '/' + obj_dir): # iterate through subject folder in training data folder
              if (str(img_path)[0] == '.'): # avoid files starting with .
                continue
              new_img_obj_dir = resized + '/' + img_dir + '/' + obj_dir
              !mkdir $new_img_obj_dir
              image = Image.open(cropped_path + '/' + img_dir + '/' + obj_dir + '/' + img_path)
              if obj_dir == 'flank':
                size = flank_size
              elif obj_dir == 'head':
                size = head_size
              else:
                size = full_size
              image = resized_image(image, size) # interpolate/resize/standardize image (using openCV)
              new_resized_path = new_img_obj_dir + '/' + img_path
              image.save(new_resized_path, format='JPEG', quality=100)
  return

def resized_image(im, desired_size):
  old_size = im.size  # old_size[0] is in (width, height) format
  ratio = float(max(desired_size))/max(old_size)
  new_size = tuple([int(x*ratio) for x in old_size])

  im = im.resize(new_size, Image.ANTIALIAS)
  
  # create a new image and paste the resized on it

  new_im = Image.new("RGB", (desired_size[0], desired_size[1]))
  new_im.paste(im, ((desired_size[0]-new_size[0])//2,
                      (desired_size[1]-new_size[1])//2))
  return new_im


def seven(paths):
  for path in paths:
    resized = path + '_resized_90/'
    ten = resized + 'seven/'
    !mkdir $ten
    for img_dir in os.listdir(resized):
      arr = [0, 0, 0]
      objs = os.listdir(resized + '/' + img_dir)
      if len(objs) == 3:
        i = 0
        for obj_dir in objs:
          list_obj = os.listdir(resized + '/' + img_dir + '/' + obj_dir)
          print(objs)
          if len(list_obj) > 5:
            arr[i] = 1
          i+=1
        if all(a == 1 for a in arr):
            for obj_dir in objs:
              list_obj = os.listdir(resized + '/' + img_dir + '/' + obj_dir)
              for img_path in list_obj:  
                ten_dir = ten + '/' + img_dir + '/'
                !mkdir $ten_dir
                ten_obj_dir = ten_dir + obj_dir + '/'
                !mkdir $ten_obj_dir
                resized_path = resized + '/' + img_dir + '/' + obj_dir + '/' + img_path
                new_path = ten_obj_dir + '/' + img_path
                !cp $resized_path $new_path

def resize_crop(train_path, test_path, weights):
  cropped([train_path, test_path], weights)
  resized([train_path, test_path])
  seven([train_path, test_path])

In [14]:
train_path = '/content/gdrive/MyDrive/leopard_NI_1/train/'
test_path = '/content/gdrive/MyDrive/leopard_NI_1/dev/'
weights = '/content/gdrive/MyDrive/Animal_Identification/yolov5/runs/train/det-610-91-def-aug-ni-rf-v3-m627/weights/last.pt'

resize_crop(train_path, test_path, weights)

mkdir: cannot create directory ‘/content/gdrive/MyDrive/leopard_NI_1/train//cropped_images/’: File exists
unknown option --source
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
mkdir: cannot create directory ‘/content/gdrive/MyDrive/leopard_NI_1/dev//cropped_images/’: File exists
unknown option --source
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
mkdir: cannot create directory ‘/content/gdrive/MyDrive/leopard_NI_1/train/_resized_90/’: File exists
mkdir: cannot create directory ‘/content/gdrive/MyDrive/leopard_NI_1/train/_resized_90//leop_0’: File exists
mkdir: cannot create directory ‘/content/gdrive/MyDrive/leopard_NI_1/train/_resized_90//leop_0/full’: File exists
mkdir: cannot create directory ‘/content/gdrive/MyDrive/leopard_NI_1/train/_resized_90//leop_0/full’: File exists
mkdir: cannot create directory ‘/content/gdrive/MyDrive/leopard_NI_1/train/_resized_90//leop_

KeyboardInterrupt: ignored